# Russian Synodal Bible (1885) and Primary Text Text Processing

## Import

### Libraries

In [1]:
import os
import json
import time
import xml.etree.ElementTree as ET
import codecs
import numpy as np
import pandas as pd 
import natasha
import re

### Definitions

In [65]:
texts = './texts/fiction/'
bibleXML = './texts/bible/synodal.xml'
bibleTXT = './texts/bible/bibliya-sinodalnyy-perevod.txt'
libCols = ['author','date','title','text']
tokenOHCO = ['title','part_num','para_num', 'sent_num', 'token_num']
bibleOHCO = ['test', 'book', 'chap', 'verse']

## Primary Texts

### Library

In [3]:
libDf = pd.DataFrame(columns = libCols)
for t in os.listdir(texts): 
    if t[-4:] == '.txt': 
        #print(t)
        info = re.match(r'(\w+)-(\d{4})-(.+).txt', t)
        with codecs.open(texts+t, 'r', encoding='windows-1251') as f: 
            textytext = f.read()
        libDf = libDf.append({
            'author': info.group(1),
            'date': int(info.group(2)), 
            'title': info.group(3), 
            'text': textytext
        }, ignore_index=True)
        
libDf = libDf.sort_values(libCols[1:3]).set_index(libCols[2])
tokenDf = libDf[[libCols[3]]]
libDf = libDf.drop(columns=[libCols[3]])
libDf

,author,date
title,,
k-zvezdam,andreev,1905
tak-bylo,andreev,1905
mat,gorkii,1906
savva-ignis-sanat,andreev,1906
iuda-iskariot,andreev,1907
zhizn-cheloveka,andreev,1907
ispoved,gorkii,1908
zhizn-nenuzhnogo-cheloveka,gorkii,1908
gorodok-okurov,gorkii,1909


### Tokens

In [4]:
tokenDf = tokenDf.text.str.split('\n\n\n', expand=True).stack().to_frame()
tokenDf.index.names = tokenOHCO[:2]
tokenDf = tokenDf.rename(columns={0:'parts'})
tokenDf = tokenDf.parts.str.split('\n\s*\n', expand=True).stack().to_frame()
tokenDf.index.names = tokenOHCO[:3]
tokenDf = tokenDf.rename(columns={0:'paras'})
tokenDf = tokenDf.paras.str.split(r'!|\?|\.|;', expand=True).stack().to_frame()
tokenDf.index.names = tokenOHCO[:4]
tokenDf = tokenDf.rename(columns={0:'sents'})
sentDf = tokenDf
tokenDf = tokenDf.sents.str.split(r'\n|\s+', expand=True).stack().to_frame()
tokenDf.index.names = tokenOHCO
tokenDf = tokenDf.rename(columns={0:'token'})
tokenDf = tokenDf.token.apply(lambda x: x.lower().strip(r"\W\|\]\}\[\{\.\'\"\?;:,<>/1234567890")).to_frame()
tokenDf = tokenDf[tokenDf['token'].str.contains(r'\w+')]
tokenDf

token
title     part_num para_num sent_num token_num              
k-zvezdam 0        1        0        1          обсерватория
                                     2                     в
                                     3                 горах
                            1        1               поздний
                                     2                 вечер
...                                                      ...
igo-voiny 0        139      94       3                 плачу
                                     4                   все
                                     5                 плачу
                                     6                   все
                                     7                 плачу

[845144 rows x 1 columns]

In [5]:
sentDf

sents
title     part_num para_num sent_num                                                   
k-zvezdam 0        0        0                                                         1
                   1        0                                    Обсерватория  в  горах
                            1                                             Поздний вечер
                            2                            Сцена представляет две комнаты
                            3         \n          первая  -  нечто вроде столовой, б...
...                                                                                 ...
igo-voiny 0        139      91         Я так люблю тебя,\n          милый, милый ты мой
                            92                                                         
                            93                                                         
                            94         \n               И все плачу, все плачу, все ...
                            95                                                       \n

[108837 rows x 1 columns]

In [6]:
sentDf = sentDf[sentDf['sents'].str.contains(r'\w+')].sents.apply(lambda x: x.lower().strip(r"\W\n\|\]\}\[\{\.\'\"\?;:,<>/1234567890")).to_frame()
sentDf#[sentDf['sents'].str.contains('r\w')]

sents
title     part_num para_num sent_num                                                   
k-zvezdam 0        0        0                                                          
                   1        0                                    обсерватория  в  горах
                            1                                             поздний вечер
                            2                            сцена представляет две комнаты
                            3         \n          первая  -  нечто вроде столовой, б...
...                                                                                 ...
igo-voiny 0        139      88         жил я "клеточкой" и\n          умру  такой  ж...
                            89        \n          но  не  могу совсем успокоиться в ...
                            90                                         дай прикоснуться
                            91         я так люблю тебя,\n          милый, милый ты мой
                            94         \n               и все плачу, все плачу, все ...

[86383 rows x 1 columns]

## Bible

### XML

### Create Bible Dictionary

### Bible

In [7]:
with open(bibleTXT, 'r', encoding='windows-1251') as f: 
    bibleText = f.readlines()

In [24]:
bibliiaDf = pd.DataFrame(bibleText).rename(columns={0:'text'}).dropna()
bibliiaDf = bibliiaDf.query(r"text != '\n'")
bibliiaDf['text'] = bibliiaDf.text.str.replace(r'\n', '')
bibliiaDf

<ipython-input-24-d42a7a2caf5f>:3: FutureWarning: The default value of regex will change from True to False in a future version.
  bibliiaDf['text'] = bibliiaDf.text.str.replace(r'\n', '')


,text
0,Библия. Синодальный перевод
1,Священное Писание
4,Перевод Библии на русский язык осуществлялся С...
10,БИБЛИЯ
12,Книги Священного Писания Ветхого и Нового Завета
...,...
191093,Обращенные из язычников.
191098,54
191101,Евреи из стран языческих.
191106,55


In [25]:
cont_line = bibliiaDf.loc[bibliiaDf.text.str.contains('Оглавление')].index[0]
OT_line = bibliiaDf.loc[bibliiaDf.text.str.contains('Книги Ветхого Завета')].index[0]
NT_line = bibliiaDf.loc[bibliiaDf.text.str.contains('Книги Нового Завета')].index[0]
end_line = bibliiaDf.loc[bibliiaDf.text.str.contains('notes')].index[0]
vz_line = bibliiaDf.loc[bibliiaDf.text.str.contains('Ветхий Завет')].index[0]
nz_line = bibliiaDf.loc[bibliiaDf.text.str.contains('Новый Завет')].index[0]

In [26]:
abbrevsDf = bibliiaDf.loc[vz_line+1:OT_line-1]
abbrevsDf[['abbr', 'title']] = abbrevsDf.text.str.extract(r'^(\[.*\]) (.*)$', expand=True)
abbrevsDf = abbrevsDf.drop(['text'], axis=1)
bibliiaDict = {
    'Ветхий Завет': dict(zip(abbrevsDf.loc[vz_line+1:nz_line-1].title.str.strip(' '), abbrevsDf.loc[vz_line+1:nz_line-1].abbr.str.strip(r'[|]'))), 
    'Новый Завет': dict(zip(abbrevsDf.loc[nz_line+1:OT_line-1].title.str.strip(' '), abbrevsDf.loc[nz_line+1:OT_line-1].abbr.str.strip(r'[|]')))
}

/home/xtra/.local/share/virtualenvs/diss-I__g6aFl/lib/python3.9/site-packages/pandas/core/frame.py:3191: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [27]:
bibliiaDf = bibliiaDf.loc[OT_line+1:end_line-1]
bibliiaDf['test'] = np.where(bibliiaDf.index<NT_line, 'OT', 'NT')
bibliiaDf

<ipython-input-27-204d3fa67429>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bibliiaDf['test'] = np.where(bibliiaDf.index<NT_line, 'OT', 'NT')


,text,test
422,[Быт.] Первая книга Моисея. Бытие,OT
426,Глава 1,OT
431,В начале сотворил Бог небо и землю.,OT
436,"Земля же была безвидна и пуста, и тьма над бе...",OT
441,И сказал Бог: да будет свет. И стал свет.,OT
...,...,...
190641,И Дух и невеста говорят: прииди! И слышавший ...,NT
190646,И я также свидетельствую всякому слышащему сл...,NT
190651,и если кто отнимет что от слов книги пророчес...,NT
190656,"Свидетельствующий сие говорит: ей, гряду скор...",NT


In [30]:
all_chaps = bibliiaDf.loc[bibliiaDf.text.str.contains(r'Глава \d')]
bibleBooks = [x for y in bibliiaDict.values() for x in y]
bookAbbrs = [val[abbr] for key, val in bibliiaDict.items() for abbr in val]
searchAbbrs = [r'^\['+val[abbr]+'\]' for key, val in bibliiaDict.items() for abbr in val]
abbrsList = "|".join(searchAbbrs)
books_index = bibliiaDf[bibliiaDf['text'].str.contains(abbrsList)].index
books_map = list(zip(books_index, bookAbbrs))
chaps_map = list(all_chaps.itertuples(name=None))

In [31]:
for ind in books_map: 
    bibliiaDf.loc[ind[0], 'book'] = ind[1]

bibliiaDf['book'] = bibliiaDf.book.ffill()
bibliiaDf = bibliiaDf.drop([x[0] for x in books_map])

/home/xtra/.local/share/virtualenvs/diss-I__g6aFl/lib/python3.9/site-packages/pandas/core/indexing.py:1599: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = infer_fill_value(value)
/home/xtra/.local/share/virtualenvs/diss-I__g6aFl/lib/python3.9/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
<ipython-input-31-fdb6dc59ab3b>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a Data

In [60]:
for ind in chaps_map: 
    bibliiaDf.loc[ind[0], 'chap'] = ind[1]
    
bibliiaDf['chap'] = bibliiaDf.chap.ffill()
bibliiaDf = bibliiaDf.drop(x[0] for x in chaps_map)
bibliiaDf['chap'] = bibliiaDf.chap.str.extract(r'(\d{1,2})$')

In [61]:
bibliiaDf['verse'] = bibliiaDf.groupby(bibleOHCO[:3]).cumcount()+1

In [67]:
bibliiaDf = bibliiaDf.set_index(bibleOHCO)
bibliiaDf

text
test book  chap verse                                                   
OT   Быт.  1    1                    В начале сотворил Бог небо и землю.
                2       Земля же была безвидна и пуста, и тьма над бе...
                3              И сказал Бог: да будет свет. И стал свет.
                4       И увидел Бог свет, что он хорош, и отделил Бо...
                5       И назвал Бог свет днем, а тьму ночью. И был в...
...                                                                  ...
NT   Откр. 22   17      И Дух и невеста говорят: прииди! И слышавший ...
                18      И я также свидетельствую всякому слышащему сл...
                19      и если кто отнимет что от слов книги пророчес...
                20      Свидетельствующий сие говорит: ей, гряду скор...
                21      Благодать Господа нашего Иисуса Христа со все...

[37343 rows x 1 columns]